# For Image Retrieval

In [2]:
### For DataFrame
import pandas as pd
import numpy as np

##### For Query Processing 
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer

### For plotting images
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from PIL import Image as PImage 
import os 
%matplotlib inline

In [89]:
tagged_data_df = pd.read_pickle("tagged_data/tag_data.pkl")
categorical_image_df = pd.read_csv("tagged_data/category_wise_image.csv")

In [4]:
category_df = pd.read_csv("tagged_data/category.csv")

### Query Processing 

In [5]:
def query_processing(sentence):
    # #sentence = input()
    # sentence = 'Fetch the images of persons and handbags'
    sentence = sentence.lower()
    words = nltk.word_tokenize(sentence)

    lemmatizer_words = []
    lemmatizer = WordNetLemmatizer()

    for word in words:
        lemmatizer_words.append(lemmatizer.lemmatize(word, wordnet.VERB))

    ## Stop words Removal
    stop_words = set(stopwords.words("english"))
    refined_words_of_lemmatizer = [word for word in lemmatizer_words if word not in stop_words]
    refined_words_of_lemmatizer = ' '.join(refined_words_of_lemmatizer)

    ## To remove puntuation marks
    final_words_lemmatizer = re.sub(r"[\W]", ' ', refined_words_of_lemmatizer)
    final_words_lemmatizer=final_words_lemmatizer.split()
    
    return final_words_lemmatizer

In [6]:
def stemmer(final_words_lemmatizer):
    stemmer_words= []
    stemmer = PorterStemmer()
    

    for word in final_words_lemmatizer:
        if word.endswith('s'):
            stemmer_words.append(stemmer.stem(word))
        else:
            stemmer_words.append(word)
            
    
    return stemmer_words

In [36]:
def custom_words_removal(object_list):

    words_to_remove = ['image', 'picture',' many', 'fetch', 'get', 'locations', 'location', 'show', 'present', 'many']

    object_list = [ i for i in object_list if i not in words_to_remove ]
    

    return object_list

In [73]:
def generate_counter(df, objects_to_detect, if_repeated_objects=False):
    
    if if_repeated_objects:
        objects_list = df.repeated_objects.values
        
    else:
        objects_list = df.objects.values

    counter = list()
    for objects in objects_list:
        c = 0
        for i in objects_to_detect:
            c += objects.count(i)

        counter.append(c)
        
    return counter

In [120]:
def generate_relevance_score(df, not_type=False, how_many=False):
    
    max_score = max(df.counter)
    
    counter = df.counter.values
    
    if how_many: 
        relevance_score = [i/max_score for i in counter]
    
    elif not_type:
        relevance_score = [1 - (i/number_of_objects) for i in counter]
        
    else:
        relevance_score = [i/number_of_objects for i in counter]
    
    return relevance_score

In [103]:
def relevance_score_sorting(df, sentence, how_many= False):
    splitted_sentence = sentence.split()

    max_score = max(tagged_data_df.counter)
    
    if how_many:
        df['relevance_score'] = generate_relevance_score(df, False, True)
        ranked_df = df.sort_values('relevance_score', ascending = False)
        
        return ranked_df
        
    
    elif 'not' in splitted_sentence:
        if len(df[df.counter == 0].index):
            df['relevance_score'] = generate_relevance_score(df, True)
            ranked_df = df.sort_values('relevance_score', ascending = False)

        else:
            return False
            print('Every image has ', obj[0], 'in it.')
    
    else:
        df['relevance_score'] = generate_relevance_score(df)
        ranked_df = df.sort_values('relevance_score', ascending = False)
        
        return ranked_df

In [153]:
def object_in_db(df):
    if df['counter'].any():
        return True

    else:
        return False

In [13]:
def show_images(PATH):

    # This is to get the directory that the program 
    # is currently running in. 
    dir_path = os.path.dirname(os.path.realpath(PATH)) 
    dir_path += "\\val2017\\"

    # for root, dirs, files in os.walk(dir_path): 
    #     for file in files: 

    for img in img_names:
        PATH = dir_path + img

        p = PATH
        print(p)
        image = mpimg.imread(p) # images are color images
        plt.gca().clear()
        plt.imshow(image);
        display.display(plt.gcf())

In [14]:
def load_class_names(file_name):
    """Returns a list of class names read from `file_name`."""
    with open(file_name, 'r') as f:
        class_names = f.read().splitlines()
    return class_names

In [15]:
def class_id_to_detect(img_names, object_list, ranked_df, category_df):
    boxes_dicts = list()
    class_id_to_detect = list()

    for img in img_names:
        boxes_dicts.append(ranked_df[ranked_df.image_name == img].repeated_objects_coordinates.values[0][0])


    for objects in object_list:
        if objects in class_names:
            class_id_to_detect.append(category_df[category_df.class_name == objects].index.values[0])

    return [class_id_to_detect, boxes_dicts]

In [16]:
def objects_frequency(obj, ranked_df):

    img_names = []
    repeated_object_len = []

    for i in range(ranked_df.shape[0]):
        for objects in obj:
            if objects in ranked_df.iloc[i][1]:
                if ranked_df.iloc[i][0] not in img_names:
                    img_names.append(ranked_df.iloc[i][0])
                    repeated_object_len.append(len(ranked_df[ranked_df.image_name == ranked_df.iloc[i][0]].repeated_objects.values[0]))
    
    return [img_names, repeated_object_len]


In [17]:
def bounding_box(ranked_df, img_names, boxes_dicts, repeated_object_len, class_id_to_detect, class_names, model_size):
    
#     model_size = ranked_df.iloc[0]['model_size']
    

    for num, img_name, boxes_dict, objects_len in zip(range(len(img_names)), img_names,
                                             boxes_dicts, repeated_object_len):
            img = Image.open('val2017/'+img_name)
            draw = ImageDraw.Draw(img)
            font = ImageFont.truetype(font='files/futur.ttf',
                                      size=(img.size[0] + img.size[1]) // 150)
            resize_factor = \
                (img.size[0] / model_size[0], img.size[1] / model_size[1])

            for cls in class_id_to_detect:
                boxes = boxes_dict[cls]
                if np.size(boxes) != 0:
                    color = np.random.permutation([np.random.randint(256), 255, 0])
                    for box in boxes:

                        xy, confidence = box[:4], box[4]
                        xy = [xy[i] * resize_factor[i % 2] for i in range(4)]
                        x0, y0 = xy[0], xy[1]
                        thickness = (img.size[0] + img.size[1]) // 200
                        for t in np.linspace(0, 1, thickness//4):
                            xy[0], xy[1] = xy[0] + t, xy[1] + t
                            xy[2], xy[3] = xy[2] - t, xy[3] - t
                            draw.rectangle(xy, outline=tuple(color))
                        text = '{}'.format(class_names[cls])

                        text_size = draw.textsize(text, font=font)
                        draw.rectangle([x0, y0 - text_size[1], x0 + text_size[0], y0], fill=tuple(color))
                        draw.text((x0, y0 - text_size[1]), text, fill='black', font=font)



            display(img)

In [131]:
def check_query_type(sentence):
    image_retrieval = False
    is_there_type = False
    location_type = False
    splitted_sentence = sentence.split()

    if sentence[:2].lower() == 'is' or sentence[:2].lower() == 'if':
        return 1
    
    elif splitted_sentence[0] == 'how' and splitted_sentence[1] == 'many' and (sentence.find('where') > 0 or sentence.find('location') > 0):
        return 2
        
    elif splitted_sentence[0] == 'how' and splitted_sentence[1] == 'many':
        return 3
    
    elif sentence.find('where') > 0 or sentence.find('location') > 0:
        return 4

    elif sentence.find('images') > 0 or sentence.find('pictures') > 0 or sentence.find('image') > 0 or sentence.find('picture') > 0:
        return 5

    else:
        return False



In [19]:
def yes_no_type(df, sentence):
    
    ranked_df = relevance_score_sorting(df, sentence)
    
    return ranked_df

In [151]:
def VQA(sentence, tagged_data_df, class_names, PATH):
    print()

    model_size = tagged_data_df.iloc[0].model_size

    splitted_sentence = sentence.split()

    query_result = check_query_type(sentence)


    if query_result:

        objects_to_detect = stemmer(custom_words_removal(query_processing(sentence)))

        number_of_objects = len(objects_to_detect)

        tagged_data_df['counter'] = generate_counter(tagged_data_df, objects_to_detect)
        
        if object_in_db(tagged_data_df):

            if query_result == 1:
                ranked_df = yes_no_type(tagged_data_df, sentence)

                if ranked_df.iloc[0].relevance_score:
                    if 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        print("Yes in the following image:")
                        print(ranked_df.iloc[0]['image_name'][:1])

                    elif 'not' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        print("Yes in the following image:")
                        for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                            print(img_name)

                    else:
                        if ranked_df.iloc[0].relevance_score:
                            print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                            print()
                            for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                                print(img_name)

                else:  
                    print("No image could be found.")

            if query_result == 2:
                tagged_data_df['counter'] = generate_counter(tagged_data_df, objects_to_detect, True)
                ranked_df = relevance_score_sorting(tagged_data_df, sentence, True)

                if ranked_df.iloc[0].relevance_score:
                    if 'or' in splitted_sentence and 'and' in splitted_sentence:
                        if splitted_sentence.index('or') < splitted_sentence.index('and'):
                            fetched_df = ranked_df[ranked_df.relevance_score!=0]
                            print("%s\t\t\t%s" %("Image Name", "Total count"))
                            for img, i in zip(fetched_df['image_name'].values,  fetched_df.counter.values):
                                print("%s\t\t\t%d" %(img, i))

                            image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                            class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                            bounding_box(ranked_df[ranked_df.relevance_score == 1], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)
                            quit()

                    if 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        fetched_df = ranked_df[ranked_df.relevance_score==1]
                        print("%s\t\t\t%s" %("Image Name", "Total count"))
                        for img, i in zip(fetched_df['image_name'].values,  fetched_df.counter.values):
                            print("%s\t\t\t%d" %(img, i))
                        image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                        class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                        bounding_box(ranked_df[ranked_df.relevance_score == 1], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)



                    elif 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score != 1:
                        print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                        print()
                        for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                            print(img_name)

                        image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                        class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                        bounding_box(ranked_df[ranked_df.relevance_score == 1], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)


                    else:
                        fetched_df = ranked_df[ranked_df.relevance_score==1]
                        print("%s\t\t\t%s" %("Image Name", "Total count"))
                        for img, i in zip(fetched_df['image_name'].values,  fetched_df.counter.values):
                            print("%s\t\t\t%d" %(img, i))

                        image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                        class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                        bounding_box(ranked_df[ranked_df.relevance_score != 0], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)


                else:  
                    print("No image could be found.")

            if query_result == 3:
                tagged_data_df['counter'] = generate_counter(tagged_data_df, objects_to_detect, True)
                ranked_df = relevance_score_sorting(tagged_data_df, sentence, True)

                if ranked_df.iloc[0].relevance_score:
                    if 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        fetched_df = ranked_df[ranked_df.relevance_score==1]
                        print("%s\t\t\t%s" %("Image Name", "Total count"))
                        for img, i in zip(fetched_df['image_name'].values,  fetched_df.counter.values):
                            print("%s\t\t\t%d" %(img, i))


                    elif 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score != 1:
                        print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                        print()
                        for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                            print(img_name)

                    else:
                       fetched_df = ranked_df[ranked_df.relevance_score==1]
                        print("%s\t\t\t%s" %("Image Name", "Total count"))
                        for img, i in zip(fetched_df['image_name'].values,  fetched_df.counter.values):
                            print("%s\t\t\t%d" %(img, i))
                else:  
                    print("No image could be found.")




            if query_result == 4:
                ranked_df = relevance_score_sorting(tagged_data_df, sentence)

                if ranked_df.iloc[0].relevance_score:
                    if 'or' in splitted_sentence and 'and' in splitted_sentence:
                        if splitted_sentence.index('or') < splitted_sentence.index('and'):
                            print("Yes in the following image:")
                            for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                                print(img_name)

                            image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                            class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                            bounding_box(ranked_df[ranked_df.relevance_score != 0], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)
                            quit()

                    if 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        print("Yes in the following image:")
                        image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                        class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                        bounding_box(ranked_df[ranked_df.relevance_score == 1], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)

                    elif 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score != 1:
                        print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                        image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score >0])
                        class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score >0], category_df)
                        bounding_box(ranked_df[ranked_df.relevance_score > 0], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)

                    else:
                        print("Yes in the following image:")
                        image_names, repeated_object_len = objects_frequency(objects_to_detect, ranked_df[ranked_df.relevance_score == 1])
                        class_ids, boxes_dict= class_id_to_detect(image_names, objects_to_detect, ranked_df[ranked_df.relevance_score == 1], category_df)
                        bounding_box(ranked_df[ranked_df.relevance_score != 0], image_names, boxes_dict, repeated_object_len, class_ids, class_names, model_size)

                else:  
                    print("No image could be found.")


            if query_result == 5:
                ranked_df = relevance_score_sorting(tagged_data_df, sentence)

                if 'not' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        print("Following are the relevant images:")
                        for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                            print(img_name)

                elif ranked_df.iloc[0].relevance_score:
                    if 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score == 1:
                        print("Following are the relevant images:")
                        for img in ranked_df[ranked_df.relevance_score==1]['image_name'].values:
                            print(img)


                    elif 'and' in splitted_sentence and ranked_df.iloc[0].relevance_score != 1:
                        print('Sorry, there is no image with all the objects present in it. Only images with either of the objects present in it:')
                        print()
                        for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:

                            print(img_name)

                    else:
                        print("Following are the relevant images:")
                        for img_name in ranked_df[ranked_df.relevance_score != 0]['image_name'].values:
                            print(img_name)

                else: 
                    print("No image could be found.")
        
        else:
            print("'",obj[0],"'","can't be found anywhere in the database.")
        
        
    else:
        print('Sorry!! The query could not be processed. RETRY!!!!')

In [152]:
PATH = 'val2017'

class_names = load_class_names('files/coco.names')

if __name__ == "__main__":
    sentence = input("Enter your query: ")
    VQA(sentence, tagged_data_df, class_names, PATH)

Enter your query: Fetch images of persons and cars.

Yes in the following image:
city.jpg
bike3.jpg
bike2.jpg


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

labelencoder_class_name = LabelEncoder()
labelencoder_category_name = LabelEncoder()

category_df2.class_name = labelencoder_class_name.fit_transform(category_df2.class_name)
category_df2.category_name = labelencoder_category_name.fit_transform(category_df2.category_name)
X = np.array(category_df2.class_name).reshape(-1, 1)

test = pd.DataFrame({'class_name': category_df.class_name, 'class_name_vector': category_df2.class_name, 'category_name': category_df.category_name, 'category_name_vector': category_df2.category_name})
category_df2

In [14]:
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, category_df2.category_name)

NameError: name 'DecisionTreeClassifier' is not defined

In [53]:
X_test = np.array(category_df2.class_name[:10]).reshape(-1, 1)
X_test

array([[48],
       [ 9],
       [18],
       [43],
       [ 0],
       [16],
       [73],
       [74],
       [11],
       [72]])

In [54]:
pred = clf.predict(X_test)

In [55]:
pred

array([6, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [56]:

print(labelencoder_category_name.inverse_transform(pred))

['person' 'transport' 'transport' 'transport' 'transport' 'transport'
 'transport' 'transport' 'transport' 'transport']


In [57]:
for i in range(len(pred)):
    print(pred[i], y_test[i])

NameError: name 'y_test' is not defined

In [ ]:
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display